In [1]:
import cv2
import numpy as np
import math

# Load and preprocess the image
img = cv2.imread(r'C:\Users\ashut\Documents\GitHub\ProjectMajor\diagram_segment.png')

if img is None:
    print("Image not found or path is incorrect!")
    exit()

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 1)
edges = cv2.Canny(blurred, 50, 150)

cv2.imshow("Edges", edges)  # See if edges are being detected properly

# Find contours and hierarchy
contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

if len(contours) == 0:
    print("No contours found!")
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    exit()

if hierarchy is None:
    print("No hierarchy returned!")
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    exit()

# simplify the hierarchy array
hierarchy = hierarchy[0]

shapes = []

# Loop through contours
for i, cnt in enumerate(contours):

    epsilon = 0.02 * cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, epsilon, True)
    vertices = len(approx)

    M = cv2.moments(cnt)
    if M['m00'] != 0:
        cx = int(M['m10'] / M['m00'])
        cy = int(M['m01'] / M['m00'])
    else:
        cx, cy = 0, 0
    center = (cx, cy)

    area = cv2.contourArea(cnt)

    side_lengths = []
    for j in range(vertices):
        p1 = approx[j][0]
        p2 = approx[(j + 1) % vertices][0]
        dist = math.hypot(p2[0] - p1[0], p2[1] - p1[1])
        side_lengths.append(dist)

    angles = []
    for j in range(vertices):
        p0 = approx[j - 1][0]
        p1 = approx[j][0]
        p2 = approx[(j + 1) % vertices][0]

        v1 = (p0[0] - p1[0], p0[1] - p1[1])
        v2 = (p2[0] - p1[0], p2[1] - p1[1])

        dot_prod = v1[0]*v2[0] + v1[1]*v2[1]
        mag1 = math.hypot(v1[0], v1[1])
        mag2 = math.hypot(v2[0], v2[1])

        if mag1 == 0 or mag2 == 0:
            angle = 0
        else:
            cos_theta = dot_prod / (mag1 * mag2)
            cos_theta = max(min(cos_theta, 1), -1)
            angle = math.acos(cos_theta) * 180 / math.pi

        angles.append(angle)

    shape = "Unknown"
    
    if vertices == 3:
        shape = "Triangle"
    elif vertices == 4:
        sides_equal = all(abs(side_lengths[k] - side_lengths[0]) < 5 for k in range(1, 4))
        angles_close_90 = all(abs(a - 90) < 10 for a in angles)

        if sides_equal and angles_close_90:
            shape = "Square"
        elif angles_close_90:
            shape = "Rectangle"
        else:
            shape = "Quadrilateral"
    elif vertices > 4:
        (x, y), radius = cv2.minEnclosingCircle(cnt)
        circularity = 4 * math.pi * area / (cv2.arcLength(cnt, True) ** 2)

        if circularity > 0.8:
            shape = "Circle"
        else:
            shape = f"{vertices}-gon"

    #print(f"Detected: {shape}")
    print(shape)
    shapes.append({
        'index': i,
        'shape': shape,
        'center': center,
        'area': area,
        'parent': hierarchy[i][3],
        'contour': approx,
        'sides': side_lengths,
        'angles': angles
    })


'''  inscribed shape formatter '''

for s in shapes:
    if s['parent'] != -1:
        parent = shapes[s['parent']]
        dx = abs(s['center'][0] - parent['center'][0])
        dy = abs(s['center'][1] - parent['center'][1])
        area_ratio = s['area'] / parent['area']
        print(f"Checking: {s['shape']} inside {parent['shape']} -> dx: {dx}, dy: {dy}, area_ratio: {area_ratio}")
        if dx < 10 and dy < 10:
            if 0.3 < area_ratio < 0.7:
                print(f"{s['shape']} is inscribed in {parent['shape']}")

for s in shapes:
    cv2.drawContours(img, [s['contour']], -1, (0, 255, 0), 2)
    cv2.putText(img, s['shape'], s['center'], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
cv2.imshow('shapes', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


6-gon
6-gon
7-gon
7-gon
6-gon
6-gon
8-gon
Square
Square
8-gon
8-gon
10-gon
10-gon
Circle
Circle
8-gon
6-gon
6-gon
8-gon
8-gon
Circle
Circle
9-gon
6-gon
6-gon
Checking: Square inside Square -> dx: 0, dy: 0, area_ratio: 0.9962462462462462
Checking: 8-gon inside 8-gon -> dx: 0, dy: 0, area_ratio: 0.9965286580728202
Checking: 10-gon inside 8-gon -> dx: 18, dy: 30, area_ratio: 0.013835887199358108
Checking: 10-gon inside 10-gon -> dx: 0, dy: 0, area_ratio: 0.983026874115983
Checking: Circle inside Circle -> dx: 0, dy: 0, area_ratio: 0.8987341772151899
Checking: 6-gon inside 6-gon -> dx: 0, dy: 0, area_ratio: 0.9966801495840647
Checking: 8-gon inside 6-gon -> dx: 79, dy: 31, area_ratio: 0.017152264634940084
Checking: 8-gon inside 8-gon -> dx: 0, dy: 0, area_ratio: 0.9776785714285714
Checking: Circle inside 8-gon -> dx: 2, dy: 3, area_ratio: 0.3070776255707763
Circle is inscribed in 8-gon
Checking: Circle inside Circle -> dx: 0, dy: 0, area_ratio: 0.9553903345724907
